In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
import matplotlib.pyplot as plt
import re
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))
import string

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import log_loss
from sklearn.metrics import zero_one_loss
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import balanced_accuracy_score

from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

pd.set_option('display.max_colwidth',100)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
df = pd.read_csv('/content/drive/My Drive/MCM_News_Aggregator/uci-news-aggregator.csv',usecols=['TITLE', 'CATEGORY','PUBLISHER'])
df.head(10)

,TITLE,PUBLISHER,CATEGORY
0,"Fed official says weak data caused by weather, should not slow taper",Los Angeles Times,b
1,Fed's Charles Plosser sees high bar for change in pace of tapering,Livemint,b
2,US open: Stocks fall after Fed official hints at accelerated tapering,IFA Magazine,b
3,"Fed risks falling 'behind the curve', Charles Plosser says",IFA Magazine,b
4,Fed's Plosser: Nasty Weather Has Curbed Job Growth,Moneynews,b
5,Plosser: Fed May Have to Accelerate Tapering Pace,NASDAQ,b
6,Fed's Plosser: Taper pace may be too slow,MarketWatch,b
7,Fed's Plosser expects US unemployment to fall to 6.2% by the end of 2014,FXstreet.com,b
8,US jobs growth last month hit by weather:Fed President Charles Plosser,Economic Times,b
9,ECB unlikely to end sterilisation of SMP purchases - traders,Interactive Investor,b


In [4]:
from keras.utils.np_utils import to_categorical

#I do aspire here to have balanced classes
num_of_categories = 45639
shuffled = df.reindex(np.random.permutation(df.index))
e = shuffled[shuffled['CATEGORY'] == 'e'][:num_of_categories]
b = shuffled[shuffled['CATEGORY'] == 'b'][:num_of_categories]
t = shuffled[shuffled['CATEGORY'] == 't'][:num_of_categories]
m = shuffled[shuffled['CATEGORY'] == 'm'][:num_of_categories]
concated = pd.concat([e,b,t,m], ignore_index=True)
#Shuffle the dataset
concated = concated.reindex(np.random.permutation(concated.index))
concated['LABEL'] = 0

concated.loc[concated['CATEGORY'] == 'e', 'LABEL'] = 0
concated.loc[concated['CATEGORY'] == 'b', 'LABEL'] = 1
concated.loc[concated['CATEGORY'] == 't', 'LABEL'] = 2
concated.loc[concated['CATEGORY'] == 'm', 'LABEL'] = 3
print(concated['LABEL'][:10])
labels = to_categorical(concated['LABEL'], num_classes=4)
print(labels[:10])
if 'CATEGORY' in concated.keys():
    concated.drop(['CATEGORY'], axis=1)

73835     1
64738     1
17870     0
163884    3
84185     1
99879     2
151131    3
36211     0
25155     0
89092     1
Name: LABEL, dtype: int64
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]


In [5]:
concated.CATEGORY.value_counts()

t    45639
m    45639
b    45639
e    45639
Name: CATEGORY, dtype: int64

In [6]:
concated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182556 entries, 73835 to 143058
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   TITLE      182556 non-null  object
 1   PUBLISHER  182555 non-null  object
 2   CATEGORY   182556 non-null  object
 3   LABEL      182556 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 7.0+ MB


In [7]:
concated['title_publish'] = concated.TITLE + " " + concated.PUBLISHER
concated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182556 entries, 73835 to 143058
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   TITLE          182556 non-null  object
 1   PUBLISHER      182555 non-null  object
 2   CATEGORY       182556 non-null  object
 3   LABEL          182556 non-null  int64 
 4   title_publish  182555 non-null  object
dtypes: int64(1), object(4)
memory usage: 8.4+ MB


In [8]:
concated.CATEGORY.value_counts()

t    45639
m    45639
b    45639
e    45639
Name: CATEGORY, dtype: int64

In [9]:
# Drop null
concated= concated.dropna(subset=['title_publish'])

In [10]:
concated.head(5)

,TITLE,PUBLISHER,CATEGORY,LABEL,title_publish
73835,ABBOTT: Supreme Court overturns EPA's illegal greenhouse gas permitting ...,Your Houston News,b,1,ABBOTT: Supreme Court overturns EPA's illegal greenhouse gas permitting ... Your Houston News
64738,Income Inequality Drives Economic Book Into US Best-Seller List,Malaysia Sun,b,1,Income Inequality Drives Economic Book Into US Best-Seller List Malaysia Sun
17870,Paul Walker's mother 'claims his daughter's mom is an alcoholic',Daily Mail,e,0,Paul Walker's mother 'claims his daughter's mom is an alcoholic' Daily Mail
163884,"Appeal growing among kids, FDA cracks down on 'wild west' of e-cigarettes",PBS NewsHour,m,3,"Appeal growing among kids, FDA cracks down on 'wild west' of e-cigarettes PBS NewsHour"
84185,Citigroup to pay $7B in subprime mortgages probe,Memphis Commercial Appeal,b,1,Citigroup to pay $7B in subprime mortgages probe Memphis Commercial Appeal


In [11]:
symbols = [',', '"', ':', ')', '(', '-', '!', '?', '|', 
           ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', 
           '#', '*', '+', '\\', '•',  '~', '@', '£', '·', '_', 
           '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', 
           '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', 
           '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', 
           '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', 
           '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', 
           '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', 
           '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', 
           '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', 
           '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', 
           '¹', '≤', '‡', '√', ]

def clean_symbol(text):
    text = str(text)
    for symbol in symbols:
        text = text.replace(symbol, '')
    return text

# remove symbols and punctuations 
concated['title_publish'] = concated['title_publish'].apply(lambda x: clean_symbol(x))

In [12]:
# Dictionary of short form words and mispellings
short_forms_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                    "'cause": "because", "could've": "could have", "couldn't": "could not", 
                    "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", 
                    "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                    "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                    "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", 
                    "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                    "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                    "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                    "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", 
                    "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", 
                    "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", 
                    "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                    "she'll've": "she will have", "she's": "she is", "should've": "should have", 
                    "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", 
                    "that's": "that is", "there'd": "there would", "there'd've": "there would have", 
                    "there's": "there is", "here's": "here is","they'd": "they would", 
                    "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", 
                    "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", 
                    "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                    "what're": "what are",  "what's": "what is", "what've": "what have", 
                    "when's": "when is", "when've": "when have", "where'd": "where did", 
                    "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", 
                    "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                    "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have",
                    "y'all're": "you all are","y'all've": "you all have","you'd": "you would", 
                    "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", 
                    "you're": "you are", "you've": "you have"}

def clean_shortforms(text):
    clean_text = text
    for shortform in short_forms_dict.keys():
        if re.search(shortform, text):
            clean_text = re.sub(shortform, short_forms_dict[shortform], text)
    return clean_text

# fix short forms
concated['title_publish'] = concated['title_publish'].apply(lambda x: clean_shortforms(x))

In [13]:
concated['title_publish']  = concated['title_publish'] .str.replace('[^\w\s]','')# unpunctuate

In [14]:
import re
import string

def clean_text(text):
    text = re.sub("[0-9]+"," ",text)
    text = text.lower() # lowercase text
    text = re.sub(' +',' ', text)
    tokens=re.split('\W+',text)
    text1 = " ".join([word for word in tokens if word not in stop_words and word not in string.punctuation])
    return text1

In [15]:
concated['clean_title_publish'] = concated['title_publish'].apply(lambda x:clean_text(x))

In [16]:
concated.head(5)

,TITLE,PUBLISHER,CATEGORY,LABEL,title_publish,clean_title_publish
73835,ABBOTT: Supreme Court overturns EPA's illegal greenhouse gas permitting ...,Your Houston News,b,1,ABBOTT Supreme Court overturns EPAs illegal greenhouse gas permitting Your Houston News,abbott supreme court overturns epas illegal greenhouse gas permitting houston news
64738,Income Inequality Drives Economic Book Into US Best-Seller List,Malaysia Sun,b,1,Income Inequality Drives Economic Book Into US BestSeller List Malaysia Sun,income inequality drives economic book us bestseller list malaysia sun
17870,Paul Walker's mother 'claims his daughter's mom is an alcoholic',Daily Mail,e,0,Paul Walkers mother claims his daughters mom is an alcoholic Daily Mail,paul walkers mother claims daughters mom alcoholic daily mail
163884,"Appeal growing among kids, FDA cracks down on 'wild west' of e-cigarettes",PBS NewsHour,m,3,Appeal growing among kids FDA cracks down on wild west of ecigarettes PBS NewsHour,appeal growing among kids fda cracks wild west ecigarettes pbs newshour
84185,Citigroup to pay $7B in subprime mortgages probe,Memphis Commercial Appeal,b,1,Citigroup to pay 7B in subprime mortgages probe Memphis Commercial Appeal,citigroup pay b subprime mortgages probe memphis commercial appeal


In [17]:
import re
import string

def clean_text1(text):
    text = string.capwords(text, sep = None) 
    tokens=re.split('\W+',text)
    tokens = list(set(tokens))
    text1 = [lemmatizer.lemmatize(word) for word in tokens if (len(word) > 2 and len(word) < 14)]
    return text1

In [18]:
concated['clean_title_publish'] = concated['clean_title_publish'].apply(lambda x:clean_text1(x))

In [19]:
concated.head(5)

,TITLE,PUBLISHER,CATEGORY,LABEL,title_publish,clean_title_publish
73835,ABBOTT: Supreme Court overturns EPA's illegal greenhouse gas permitting ...,Your Houston News,b,1,ABBOTT Supreme Court overturns EPAs illegal greenhouse gas permitting Your Houston News,"[Epas, Greenhouse, Abbott, Court, Houston, Permitting, Supreme, Overturns, News, Illegal, Gas]"
64738,Income Inequality Drives Economic Book Into US Best-Seller List,Malaysia Sun,b,1,Income Inequality Drives Economic Book Into US BestSeller List Malaysia Sun,"[Income, List, Sun, Economic, Bestseller, Drives, Inequality, Book, Malaysia]"
17870,Paul Walker's mother 'claims his daughter's mom is an alcoholic',Daily Mail,e,0,Paul Walkers mother claims his daughters mom is an alcoholic Daily Mail,"[Mom, Daily, Claims, Daughters, Mother, Alcoholic, Walkers, Mail, Paul]"
163884,"Appeal growing among kids, FDA cracks down on 'wild west' of e-cigarettes",PBS NewsHour,m,3,Appeal growing among kids FDA cracks down on wild west of ecigarettes PBS NewsHour,"[West, Fda, Ecigarettes, Growing, Cracks, Kids, Pbs, Among, Wild, Newshour, Appeal]"
84185,Citigroup to pay $7B in subprime mortgages probe,Memphis Commercial Appeal,b,1,Citigroup to pay 7B in subprime mortgages probe Memphis Commercial Appeal,"[Subprime, Mortgages, Probe, Citigroup, Commercial, Pay, Memphis, Appeal]"


In [20]:
import re
import string

def clean_text2(text):
    text1 = " ".join([lemmatizer.lemmatize(word) for word in text if (len(word) > 2 and len(word) < 14)])
    return text1

In [21]:
concated['clean_title_publish1'] = concated['clean_title_publish'].apply(lambda x:clean_text2(x))

In [22]:
concated.head(5)

,TITLE,PUBLISHER,CATEGORY,LABEL,title_publish,clean_title_publish,clean_title_publish1
73835,ABBOTT: Supreme Court overturns EPA's illegal greenhouse gas permitting ...,Your Houston News,b,1,ABBOTT Supreme Court overturns EPAs illegal greenhouse gas permitting Your Houston News,"[Epas, Greenhouse, Abbott, Court, Houston, Permitting, Supreme, Overturns, News, Illegal, Gas]",Epas Greenhouse Abbott Court Houston Permitting Supreme Overturns News Illegal Gas
64738,Income Inequality Drives Economic Book Into US Best-Seller List,Malaysia Sun,b,1,Income Inequality Drives Economic Book Into US BestSeller List Malaysia Sun,"[Income, List, Sun, Economic, Bestseller, Drives, Inequality, Book, Malaysia]",Income List Sun Economic Bestseller Drives Inequality Book Malaysia
17870,Paul Walker's mother 'claims his daughter's mom is an alcoholic',Daily Mail,e,0,Paul Walkers mother claims his daughters mom is an alcoholic Daily Mail,"[Mom, Daily, Claims, Daughters, Mother, Alcoholic, Walkers, Mail, Paul]",Mom Daily Claims Daughters Mother Alcoholic Walkers Mail Paul
163884,"Appeal growing among kids, FDA cracks down on 'wild west' of e-cigarettes",PBS NewsHour,m,3,Appeal growing among kids FDA cracks down on wild west of ecigarettes PBS NewsHour,"[West, Fda, Ecigarettes, Growing, Cracks, Kids, Pbs, Among, Wild, Newshour, Appeal]",West Fda Ecigarettes Growing Cracks Kids Pbs Among Wild Newshour Appeal
84185,Citigroup to pay $7B in subprime mortgages probe,Memphis Commercial Appeal,b,1,Citigroup to pay 7B in subprime mortgages probe Memphis Commercial Appeal,"[Subprime, Mortgages, Probe, Citigroup, Commercial, Pay, Memphis, Appeal]",Subprime Mortgages Probe Citigroup Commercial Pay Memphis Appeal


In [23]:
concated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182555 entries, 73835 to 143058
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   TITLE                 182555 non-null  object
 1   PUBLISHER             182555 non-null  object
 2   CATEGORY              182555 non-null  object
 3   LABEL                 182555 non-null  int64 
 4   title_publish         182555 non-null  object
 5   clean_title_publish   182555 non-null  object
 6   clean_title_publish1  182555 non-null  object
dtypes: int64(1), object(6)
memory usage: 11.1+ MB


In [24]:
import gensim

#Dimension of vectors we are generating
EMBEDDING_DIM = 300

#Creating Word Vectors by Word2Vec Method (takes time...)
wv = gensim.models.Word2Vec(sentences=concated['clean_title_publish'], size=EMBEDDING_DIM, window=5, min_count=1)  

In [25]:
#vocab size
len(wv.wv.vocab)
#We have now represented each of 122248 words by a 100dim vector.

46674

In [26]:
#printing similarity index
print(wv.most_similar('English'))

[('Allay', 0.9104262590408325), ('Asharq', 0.9074191451072693), ('Urthecast', 0.9011282920837402), ('Eradication', 0.8979059457778931), ('Nurse', 0.8966658115386963), ('Praised', 0.8953779339790344), ('Kuwait', 0.8949897289276123), ('Hoarder', 0.8948409557342529), ('Colleague', 0.8929315805435181), ('Nurses', 0.8920722007751465)]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [27]:
wv['English']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-3.23147893e-01,  2.02571154e-01,  2.06312642e-01,  3.55028026e-02,
        2.30118543e-01,  2.87010610e-01, -5.55116422e-02, -2.70325452e-01,
        7.26245940e-02,  2.00905100e-01, -5.76674268e-02,  1.54419541e-01,
       -1.88722298e-01,  2.87112862e-01,  1.50215492e-01, -3.57121736e-01,
        1.31714806e-01,  1.08403368e-02,  1.92228496e-01,  2.11681396e-01,
        2.74826288e-01, -1.31771877e-01, -3.85403484e-01,  2.59671330e-01,
       -1.51135534e-01,  2.92619586e-01,  6.96093310e-03, -3.74236517e-02,
       -9.52753425e-02, -2.45178372e-01, -1.11067541e-01,  4.98759240e-01,
        7.66072646e-02, -6.54591694e-02,  6.45082444e-02, -1.24688059e-01,
        1.22932680e-01, -1.60346672e-01, -1.56258583e-01,  8.72692615e-02,
       -6.04512654e-02,  1.92760140e-01, -2.05440447e-01, -3.47689204e-02,
        3.04752380e-01,  2.18294695e-01,  1.52881265e-01, -1.59537867e-01,
        1.65656194e-01, -3.49135011e-01, -3.20001654e-02, -1.37368068e-01,
       -3.69725376e-01, -

In [28]:
#Word averaging
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)


    if not mean:
        #logging.warning("cannot compute similarity with no input %s", words)
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, clean_title_publish) for clean_title_publish in text_list])

In [29]:
# Tokenization
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [30]:
%%time
train, test = train_test_split(concated, test_size=0.3,shuffle=True)
test_tokenized = test.apply(lambda r: w2v_tokenize_text(r['clean_title_publish1']), axis=1).values
train_tokenized = train.apply(lambda r: w2v_tokenize_text(r['clean_title_publish1']), axis=1).values

CPU times: user 29.1 s, sys: 188 ms, total: 29.3 s
Wall time: 29.3 s


In [31]:
%%time
X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


CPU times: user 12.7 s, sys: 386 ms, total: 13.1 s
Wall time: 13.1 s


Log Regression

In [32]:
"""
# multi_class
multi_class = ['multinomial']

# solver
solver = ['newton-cg', 'sag', 'saga', 'lbfgs']
 
# class_weight
class_weight = ['balanced', None]

# penalty
penalty = ['l2']

# Create the random grid
random_grid = {
               'multi_class': multi_class,
               'solver': solver,
               'class_weight': class_weight,
               'penalty': penalty}
"""               

"\n# multi_class\nmulti_class = ['multinomial']\n\n# solver\nsolver = ['newton-cg', 'sag', 'saga', 'lbfgs']\n \n# class_weight\nclass_weight = ['balanced', None]\n\n# penalty\npenalty = ['l2']\n\n# Create the random grid\nrandom_grid = {\n               'multi_class': multi_class,\n               'solver': solver,\n               'class_weight': class_weight,\n               'penalty': penalty}\n"

In [33]:
"""
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
# First create the base model to tune
lrc = LogisticRegression(random_state=8)

# Definition of the random search
random_search = RandomizedSearchCV(estimator=lrc,
                                   param_distributions=random_grid,
                                   n_iter=50,
                                   scoring='accuracy',
                                   cv=3, 
                                   verbose=1, 
                                   random_state=8)

# Fit the random search model
random_search.fit(X_train_word_average, train['LABEL'])
"""

"\nfrom sklearn.model_selection import RandomizedSearchCV\nfrom sklearn.model_selection import GridSearchCV\n# First create the base model to tune\nlrc = LogisticRegression(random_state=8)\n\n# Definition of the random search\nrandom_search = RandomizedSearchCV(estimator=lrc,\n                                   param_distributions=random_grid,\n                                   n_iter=50,\n                                   scoring='accuracy',\n                                   cv=3, \n                                   verbose=1, \n                                   random_state=8)\n\n# Fit the random search model\nrandom_search.fit(X_train_word_average, train['LABEL'])\n"

In [34]:
"""
print("The best hyperparameters from Random Search are:")
print(random_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(random_search.best_score_)

"""


'\nprint("The best hyperparameters from Random Search are:")\nprint(random_search.best_params_)\nprint("")\nprint("The mean accuracy of a model with these hyperparameters is:")\nprint(random_search.best_score_)\n\n'

In [35]:
# First create the base model to tune
logreg = LogisticRegression(n_jobs=-1,solver='newton-cg',penalty='l2',multi_class='multinomial',class_weight='balanced')

# Fit the random search model
logreg.fit(X_train_word_average, train['LABEL'])

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='multinomial', n_jobs=-1,
                   penalty='l2', random_state=None, solver='newton-cg',
                   tol=0.0001, verbose=0, warm_start=False)

In [36]:
y_pred_test_log=logreg.predict(X_test_word_average)
y_pred_train_log=logreg.predict(X_train_word_average)

In [37]:
d=accuracy_score(y_pred_test_log,test.LABEL)*100
print ('Test Accuracy',d)

e=accuracy_score(y_pred_train_log,train.LABEL)*100
print ('Train Accuracy',e)

Test Accuracy 83.2472109116804
Train Accuracy 83.52114439540489


In [38]:
bac=balanced_accuracy_score(y_pred_test_log,test.LABEL)*100
c_k_s=cohen_kappa_score(y_pred_test_log,test.LABEL)*100
mc=matthews_corrcoef(y_pred_test_log,test.LABEL)*100


print(c_k_s)
print(mc)
print(bac)

77.66266630085971
77.66903588960378
83.22345921899051


In [39]:
%%time
sgd_search = SGDClassifier(n_jobs=-1, max_iter=15,alpha=0.00001)
sgd_search.fit(X_train_word_average,  train['LABEL'])

y_pred_test_sgd = sgd_search.predict(X_test_word_average)
y_pred_train_sgd = sgd_search.predict(X_train_word_average)


CPU times: user 8.84 s, sys: 36.5 ms, total: 8.88 s
Wall time: 4.66 s


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [40]:
f=accuracy_score(y_pred_test_sgd,test.LABEL)*100
print ('Test Accuracy',f)

g=accuracy_score(y_pred_train_sgd,train.LABEL)*100
print ('Train Accuracy',g)

Test Accuracy 83.09383387806525
Train Accuracy 83.42567377218518


In [41]:
print(classification_report(test.LABEL, y_pred_test_sgd))

              precision    recall  f1-score   support

           0       0.91      0.86      0.88     13657
           1       0.80      0.78      0.79     13809
           2       0.79      0.81      0.80     13669
           3       0.83      0.88      0.85     13632

    accuracy                           0.83     54767
   macro avg       0.83      0.83      0.83     54767
weighted avg       0.83      0.83      0.83     54767



In [42]:
# title-category function
def title_to_category1(title):
    title=clean_text(title)
    title=clean_symbol(title)
    title=clean_shortforms(title)
    categories = {1 : 'business', 
                  2 : 'science and technology', 
                  0 : 'entertainment', 
                  3 : 'health'}
    pridicter1 = logreg.predict(word_averaging_list(wv,w2v_tokenize_text(title)))
    return categories[pridicter1[0]]

In [43]:
title_to_category1('12 family getaways in Ireland this summer')

'health'

In [44]:
# title-category function
def title_to_category2(title):
    title=clean_text(title)
    title=clean_symbol(title)
    title=clean_shortforms(title)
    categories = {1 : 'business', 
                  2 : 'science and technology', 
                  0 : 'entertainment', 
                  3 : 'health'}
    pridicter2 = sgd_search.predict(word_averaging_list(wv,w2v_tokenize_text(title)))
    return categories[pridicter2[0]]

In [45]:
title_to_category2('Whole four-star hotel up for grabs in charity raffle')

'health'